# Supporting code and data for "Automatic Bug Minimization in Proof Assistants: A Case Study in Coq"

In [1]:
%matplotlib inline

import os
import sys
print(f'Python {sys.version}')

import IPython
from IPython.core.display import display, HTML
print(f'IPython {IPython.__version__}')

print('\nLibraries:\n')

import csv
print(f'csv {csv.__version__}')

import matplotlib
import matplotlib.pyplot as plt
print(f'matplotlib {matplotlib.__version__}')

import numpy as np
print(f'numpy {np.__version__}')

import pandas as pd
from pandas.plotting import register_matplotlib_converters
print(f'pandas {pd.__version__}')

import re
print(f're {re.__version__}')

import requests
print(f'requests {requests.__version__}')

Python 3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]
IPython 7.13.0

Libraries:

csv 1.0
matplotlib 3.1.2
numpy 1.23.1
pandas 1.4.3
re 2.2.1
requests 2.27.1


## Data collection

We use the GitHub GraphQL API because it allows fetching only the information we need, and at a much faster rate (we can get up to 100 nodes in a single request). Getting all the objects of a certain type requires then to repeat the request to go through all the pages of results.

You need to provide a personal `api_token` if you want to get fresh data from GitHub. Otherwise, this notebook will skip the data collection step and load the CSV files from the local filesystem.

In [2]:
api_token = ''

In [3]:
def requestAllPages(query,rows_and_next_variables,filename,columns):
  if api_token == '':
    return
  headers = {'Authorization': f'token {api_token}'}
  url = 'https://api.github.com/graphql'
  rows, variables = rows_and_next_variables(None)
  while len(variables)>0:
    json = {'query':query,'variables':variables.pop()}
    r = requests.post(url=url, json=json, headers=headers)
    if r.status_code == 403:
      print('Unauthorized request:')
      print(json)
    r.raise_for_status() # Abort if unsuccessful request
    new_rows, next_variables = rows_and_next_variables(r.json()['data'])
    rows += new_rows
    variables += next_variables
  if len(rows) > 0:
    with open(filename, 'w', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(columns)
      # deduplicate rows
      rows = set(map(tuple, rows))
      # sort rows, for ease of tracking future updates in git
      rows = sorted(rows, key=(lambda r: (r[1], r[0], r)))
      writer.writerows(rows)

We search all PRs where CI minimization was proposed (excluding those authored by Jason Gross, which were mostly to debug the minimizer) and we retrieve all the comments from coqbot-app to know what happened. We only keep the first 15 lines of each comment, to reduce the size of the CSV file, because these lines will contain all the information we need.

Make sure to uncomment the last line and to provide an `api_token` to re-run this.

In [4]:
def fetch_pr_comments():

  query = """
    query commentQuery($number: Int!, $single: Boolean!, $prCursor: String, $commentCursor: String) {
      search(query: "repo:coq/coq coqbot ci minimize", type:ISSUE, first: 10, after: $prCursor) @skip (if: $single) {
        pageInfo {
          endCursor
          hasNextPage
        }
        nodes {
          ... pullRequest
        }
      }
      repository(owner: "coq", name: "coq") @include (if: $single) {
        pullRequest(number: $number) {
          ... pullRequest
        }
      }
    }

    fragment pullRequest on PullRequest {
      number
      author { login }
      comments(first: 50, after: $commentCursor) {
        pageInfo {
          endCursor
          hasNextPage
        }
        nodes {
          createdAt
          author { login }
          bodyText
          databaseId
        }
      }
    }
  """

  def treat_pr(pr):
    rows, variables = [], []
    number = pr['number']
    if pr['author']['login'] != 'JasonGross':
      for comment in pr['comments']['nodes']:
        if comment['author']['login'] == "coqbot-app":
          date = pd.to_datetime(comment['createdAt']).tz_localize(None)
          body = '\\n'.join(comment['bodyText'].split('\n')[:15])
          rows.append([comment['databaseId'],number,date,body])
    if pr['comments']['pageInfo']['hasNextPage']:
      variables += [{
          'single':True,
          'number':number,
          'commentCursor':pr['comments']['pageInfo']['endCursor']
      }]
    return rows, variables

  def rows_and_next_variables(data):
    if data is None:
      return [], [{'single':False,'number':0}]
    else:
      if 'search' in data:
        prs = data['search']
        rows, variables = [], []
        for pr in prs['nodes']:
          if 'number' in pr:
            new_rows, new_variables = treat_pr(pr)
            rows += new_rows
            variables += new_variables
        if prs['pageInfo']['hasNextPage']:
          variables += [{
              'single':False,
              'number':0,
              'prCursor':prs['pageInfo']['endCursor']
          }]
        return rows, variables
      else:
        return treat_pr(data['repository']['pullRequest'])

  requestAllPages(
      query,
      rows_and_next_variables,
      'pr_comments.csv',
      ['id','number','date','body']
  )

# fetch_pr_comments()

## Data processing

We retrieve the saved dataset from disk.

In [5]:
coqbot_comments = pd.read_csv('pr_comments.csv',parse_dates=['date'],index_col=0)

The data retrieved from GitHub can contain duplicated entries (same ID).

In [6]:
coqbot_comments[coqbot_comments.index.duplicated()]

,number,date,body
id,,,


We remove them:

In [7]:
coqbot_comments = coqbot_comments[~coqbot_comments.index.duplicated(keep='first')]

### Matching CI minimize comments

We look for comments marking the beginning and the end of the minimization. We only keep the last run for each pull request and minimized project to avoid double counting minimization examples.

In [8]:
coqbot_comments = coqbot_comments.assign(targets = coqbot_comments['body'].str.extract(r'I (?:have initiated|am now running) minimization at commit [a-z0-9]* (?:for the suggested targets? |on requested targets? )(?P<targets>[^.]+)'))
coqbot_comments_with_targets = coqbot_comments[~coqbot_comments['targets'].isna()]
coqbot_comments_with_targets

,number,date,body,targets
id,,,,
886229983,11966,2021-07-25 16:58:47,I have initiated minimization at commit 31cf6f...,"ci-metacoq, ci-rewriter, ci-vst as requested"
985697650,11966,2021-12-03 17:25:58,I have initiated minimization at commit fed309...,ci-vst as requested
864456494,12493,2021-06-19 19:57:55,I have initiated minimization at commit 01ae7b...,"ci-bedrock2, ci-color as requested"
883774436,12493,2021-07-20 23:39:10,I have initiated minimization at commit 8b0be5...,"ci-bedrock2, ci-color as requested"
864662718,12512,2021-06-21 01:37:28,I have initiated minimization at commit 39ce95...,ci-rewriter as requested
...,...,...,...,...
1196917668,16311,2022-07-27 15:34:05,I have initiated minimization at commit 6a8511...,"ci-bedrock2, ci-fiat_crypto_legacy, ci-menhir,..."
1186439325,16322,2022-07-17 08:04:42,I am now running minimization at commit fd8c81...,ci-category_theory
1190283172,16332,2022-07-20 13:24:05,I have initiated minimization at commit ea8e79...,"ci-color, ci-compcert, ci-coqprime, ci-fiat_cr..."


In [9]:
targets = coqbot_comments_with_targets['targets'].str.extractall(r'(?P<target>ci-[^,\s]*)')
# Exclude accidental runs on the Coq bug minimizer itself
targets = targets[targets['target'] != 'ci-coq_tools']
# ci-metacoq build system was responsible for many failures, if we exclude it we boost our success score from 75 to 79%
# targets = targets[targets['target'] != 'ci-metacoq']
minimization_started_comments = targets.join(coqbot_comments_with_targets).sort_values('date').drop_duplicates(subset=['target','number'], keep='last').set_index(['target','number'])[['date']]
minimization_started_comments

,,date
target,number,
ci-mathcomp,13969,2021-05-24 12:34:06
ci-equations,13969,2021-05-26 14:10:29
ci-fourcolor,13969,2021-05-26 14:10:29
ci-iris,13969,2021-05-26 14:10:29
ci-perennial,13969,2021-05-26 14:10:29
...,...,...
ci-mtac2,16311,2022-07-27 15:34:05
ci-menhir,16311,2022-07-27 15:34:05
ci-fiat_crypto_legacy,16311,2022-07-27 15:34:05


For successful minimization runs (i.e., runs that produced a minimized file, and were not automatically restarted after being interrupted by a timeout), we extract information from the headers of the minimized files, such as the expected `coqc` runtime on this file, the initial number of lines, and the final number of lines, or if any module couldn't be inlined.

In [10]:
minimization_success_comments = coqbot_comments[coqbot_comments['body'].str.startswith('Minimized File') & ~coqbot_comments['body'].str.contains('interrupted by timeout, being automatically continued')]
minimization_success_comments = minimization_success_comments.assign(
    runtime = minimization_success_comments['body'].str.extract(r'Expected coqc runtime on this file: ([0-9\.]+) sec').astype(float),
    initial_size = minimization_success_comments['body'].str.extract(r'from original input, then from ([0-9]+) lines').astype(float),
    final_size = minimization_success_comments['body'].str.extract(r'to ([0-9]+) lines \*\)').astype(float),
    inline_failure = minimization_success_comments['body'].str.contains('could not be inlined'),
    truncated = minimization_success_comments['body'].str.contains('Minimized Coq File \(truncated')
)
minimization_success_comments

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,,
886230665,11966,2021-07-25 17:04:47,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
886240171,11966,2021-07-25 18:26:22,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,493.0,141.0,False,False
886395867,11966,2021-07-26 05:49:12,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,2095.0,2464.0,False,True
986135923,11966,2021-12-04 23:37:37,Minimized File /github/workspace/builds/coq/co...,NaN,1.011,2430.0,1320.0,True,True
864457135,12493,2021-06-19 20:03:50,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...
1190508285,16332,2022-07-20 16:39:34,Minimized File /github/workspace/builds/coq/co...,NaN,1.357,139.0,100.0,True,False
1190898852,16332,2022-07-21 00:22:10,Minimized File /github/workspace/builds/coq/co...,NaN,0.409,2073.0,392.0,False,False
1191746703,16337,2022-07-21 17:21:49,Minimized File /github/workspace/builds/coq/co...,NaN,0.600,197.0,147.0,False,False


There are a few files for which we couldn't retrieve any info from the headers (or with a header with absurd data). These actually correspond to files where the minimizer was not able to reproduce any error message and thus did not really produce a reduced case. Thus, we should count this together with the minimization failures:

In [11]:
minimization_false_successes = minimization_success_comments[minimization_success_comments['initial_size'].isna() | (minimization_success_comments['initial_size'] == 0)]
minimization_success_comments = minimization_success_comments[~minimization_success_comments['initial_size'].isna() & (minimization_success_comments['initial_size'] > 0)]
minimization_false_successes

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,,
886230665,11966,2021-07-25 17:04:47,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
864457135,12493,2021-06-19 20:03:50,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
883776198,12493,2021-07-20 23:44:43,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
909643095,12512,2021-08-31 21:12:31,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
857796568,13072,2021-06-09 15:21:58,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
857826783,13072,2021-06-09 15:56:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
870663613,13107,2021-06-29 14:43:55,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
870663887,13107,2021-06-29 14:44:15,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
870664130,13107,2021-06-29 14:44:32,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,0.0,638.0,False,False


For these reduced test cases, we also compute the total number of removed lines by aggregating the information found in the headers about the number of lines removed at each step.

In [12]:
line_reduction = minimization_success_comments['body'].str.extractall(r'then from (?P<reduced_from>[0-9]+) lines to (?P<reduced_to>[0-9]+) lines')
line_reduction = line_reduction.assign(total_line_reduction = line_reduction['reduced_from'].astype(int) - line_reduction['reduced_to'].astype(int))
line_reduction = line_reduction.groupby(level=[0]).sum()
minimization_success_comments = minimization_success_comments.join(line_reduction['total_line_reduction'])
minimization_success_comments

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
id,,,,,,,,,,
886240171,11966,2021-07-25 18:26:22,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,493.0,141.0,False,False,620
886395867,11966,2021-07-26 05:49:12,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,2095.0,2464.0,False,True,2355
986135923,11966,2021-12-04 23:37:37,Minimized File /github/workspace/builds/coq/co...,NaN,1.011,2430.0,1320.0,True,True,1966
883873890,12493,2021-07-21 04:16:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,533.0,857.0,False,False,1222
864825030,12512,2021-06-21 08:10:55,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,2099.0,1618.0,False,True,1176
...,...,...,...,...,...,...,...,...,...,...
1190508285,16332,2022-07-20 16:39:34,Minimized File /github/workspace/builds/coq/co...,NaN,1.357,139.0,100.0,True,False,117
1190898852,16332,2022-07-21 00:22:10,Minimized File /github/workspace/builds/coq/co...,NaN,0.409,2073.0,392.0,False,False,1885
1191746703,16337,2022-07-21 17:21:49,Minimized File /github/workspace/builds/coq/co...,NaN,0.600,197.0,147.0,False,False,123


In [13]:
minimization_failure_comments = pd.concat([coqbot_comments[coqbot_comments['body'].str.startswith('Error: Could not minimize file')], minimization_false_successes])
minimization_failure_comments

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,,
851692477,12743,2021-05-31 21:33:24,Error: Could not minimize file /github/workspa...,NaN,NaN,NaN,NaN,NaN,NaN
851702877,12743,2021-05-31 22:22:52,Error: Could not minimize file /github/workspa...,NaN,NaN,NaN,NaN,NaN,NaN
857804076,13072,2021-06-09 15:30:20,Error: Could not minimize file (from ci-metac...,NaN,NaN,NaN,NaN,NaN,NaN
857828059,13072,2021-06-09 15:57:55,Error: Could not minimize file /github/workspa...,NaN,NaN,NaN,NaN,NaN,NaN
857885217,13072,2021-06-09 17:21:12,Error: Could not minimize file /github/workspa...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
965653376,15128,2021-11-10 19:08:04,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
1013700274,15487,2022-01-15 15:26:47,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
1017939088,15518,2022-01-20 21:22:16,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True


We aggregate successful and failed end minimization comments and only keep the last run for each project and pull request:

In [14]:
minimization_finished_comments = pd.concat([minimization_success_comments.assign(success=True), minimization_failure_comments.assign(success=False)])
targets = minimization_finished_comments['body'].str.extract(r'(?P<target>ci-[^)]*)')
# Exclude accidental runs on the Coq bug minimizer itself
targets = targets[targets['target'] != 'ci-coq_tools']
minimization_finished_comments = targets.join(minimization_finished_comments).sort_values('date').drop_duplicates(subset=['target','number'], keep='last').set_index(['target','number'])#[['date','success','runtime']]
minimization_finished_comments

,,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success
target,number,,,,,,,,,,
ci-mathcomp,13969,2021-05-24 13:13:51,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,4170.0,109.0,False,False,4146.0,True
ci-iris,13969,2021-05-26 14:13:55,Error: Could not minimize file (from ci-iris)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
ci-equations,13969,2021-05-26 14:14:19,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False
ci-fourcolor,13969,2021-05-26 14:15:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False
ci-perennial,13969,2021-05-26 14:16:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,1726.0,3.0,False,False,3455.0,True
...,...,...,...,...,...,...,...,...,...,...,...
ci-menhir,16311,2022-07-27 15:43:23,Minimized File /github/workspace/builds/coq/co...,NaN,0.147,407.0,51.0,False,False,397.0,True
ci-fiat_crypto_legacy,16311,2022-07-27 15:46:50,Minimized File /github/workspace/builds/coq/co...,NaN,0.255,709.0,21.0,False,False,693.0,True
ci-mtac2,16311,2022-07-27 16:03:20,Minimized File /github/workspace/builds/coq/co...,NaN,0.259,519.0,204.0,False,False,506.0,True


We match these with the comments corresponding to the beginning of the minimization to compute the minimization duration:

In [15]:
minimization_pairs = minimization_started_comments.join(minimization_finished_comments,lsuffix='_start',rsuffix='_end')
minimization_pairs = minimization_pairs.assign(duration=(minimization_pairs['date_end'] - minimization_pairs['date_start']).dt.seconds)
minimization_pairs

,,date_start,date_end,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success,duration
target,number,,,,,,,,,,,,
ci-mathcomp,13969,2021-05-24 12:34:06,2021-05-24 13:13:51,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,4170.0,109.0,False,False,4146.0,True,2385.0
ci-equations,13969,2021-05-26 14:10:29,2021-05-26 14:14:19,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,230.0
ci-fourcolor,13969,2021-05-26 14:10:29,2021-05-26 14:15:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,298.0
ci-iris,13969,2021-05-26 14:10:29,2021-05-26 14:13:55,Error: Could not minimize file (from ci-iris)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,206.0
ci-perennial,13969,2021-05-26 14:10:29,2021-05-26 14:16:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,1726.0,3.0,False,False,3455.0,True,339.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ci-mtac2,16311,2022-07-27 15:34:05,2022-07-27 16:03:20,Minimized File /github/workspace/builds/coq/co...,NaN,0.259,519.0,204.0,False,False,506.0,True,1755.0
ci-menhir,16311,2022-07-27 15:34:05,2022-07-27 15:43:23,Minimized File /github/workspace/builds/coq/co...,NaN,0.147,407.0,51.0,False,False,397.0,True,558.0
ci-fiat_crypto_legacy,16311,2022-07-27 15:34:05,2022-07-27 15:46:50,Minimized File /github/workspace/builds/coq/co...,NaN,0.255,709.0,21.0,False,False,693.0,True,765.0


For the successful cases, there are other data that we can analyze:

In [16]:
successful_minimization_pairs = minimization_pairs[minimization_pairs['success'] == True][['date_start','duration','runtime','initial_size','final_size','inline_failure','truncated','total_line_reduction']]
successful_minimization_pairs

,,date_start,duration,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
target,number,,,,,,,,
ci-mathcomp,13969,2021-05-24 12:34:06,2385.0,NaN,4170.0,109.0,False,False,4146.0
ci-perennial,13969,2021-05-26 14:10:29,339.0,NaN,1726.0,3.0,False,False,3455.0
ci-quickchick,13969,2021-05-26 14:10:29,534.0,NaN,1119.0,66.0,False,False,1074.0
ci-interval,13895,2021-05-27 22:11:13,2064.0,NaN,3383.0,241.0,False,False,3249.0
ci-coq_performance_tests,12743,2021-06-01 17:17:15,19642.0,NaN,1113.0,312.0,False,False,801.0
...,...,...,...,...,...,...,...,...,...
ci-mtac2,16311,2022-07-27 15:34:05,1755.0,0.259,519.0,204.0,False,False,506.0
ci-menhir,16311,2022-07-27 15:34:05,558.0,0.147,407.0,51.0,False,False,397.0
ci-fiat_crypto_legacy,16311,2022-07-27 15:34:05,765.0,0.255,709.0,21.0,False,False,693.0


Number of pull requests these minimization pairs come from:

In [17]:
len(minimization_pairs.index.to_frame()[['number']].drop_duplicates())

74

Now we save the results of data analysis

In [ ]:
minimization_pairs.to_csv('minimization_pairs.csv'pd.write_csv('pr_comments.csv',parse_dates=['date'],index_col=0)

### Results

#### RQ1: How often does the minimizer produce a reduced test case?

Proportion of the time the minimizer successfully produced a reduced file:

In [18]:
len(successful_minimization_pairs)/len(minimization_pairs)

0.848

Let's take a look at the failed runs. First, the runs for which no finish comment was found:

In [19]:
minimization_pairs[minimization_pairs['date_end'].isna()]

date_start date_end body targets  \
target                number                                             
ci-vst                14746  2021-08-06 14:10:50      NaT  NaN     NaN   
                      14777  2021-08-13 12:21:08      NaT  NaN     NaN   
ci-metacoq            14819  2021-09-02 13:49:41      NaT  NaN     NaN   
ci-bedrock2           15128  2021-11-06 00:58:32      NaT  NaN     NaN   
ci-fiat_crypto_legacy 15178  2021-11-12 17:27:33      NaT  NaN     NaN   

                              runtime  initial_size  final_size  \
target                number                                      
ci-vst                14746       NaN           NaN         NaN   
                      14777       NaN           NaN         NaN   
ci-metacoq            14819       NaN           NaN         NaN   
ci-bedrock2           15128       NaN           NaN         NaN   
ci-fiat_crypto_legacy 15178       NaN           NaN         NaN   

                             inline_failure truncated  total_line_reduction  \
target                number                                                  
ci-vst                14746             NaN       NaN                   NaN   
                      14777             NaN       NaN                   NaN   
ci-metacoq            14819             NaN       NaN                   NaN   
ci-bedrock2           15128             NaN       NaN                   NaN   
ci-fiat_crypto_legacy 15178             NaN       NaN                   NaN   

                             success  duration  
target                number                    
ci-vst                14746      NaN       NaN  
                      14777      NaN       NaN  
ci-metacoq            14819      NaN       NaN  
ci-bedrock2           15128      NaN       NaN  
ci-fiat_crypto_legacy 15178      NaN       NaN

In the last four cases, there was an infinite loop and the minimizer had to be stopped manually.

Next, the runs for which minimization concluded with an explicit error:

In [20]:
minimization_pairs[(minimization_pairs['success'] == False) & minimization_pairs['body'].str.startswith('Error: Could not minimize file')]

date_start            date_end  \
target              number                                           
ci-iris             13969  2021-05-26 14:10:29 2021-05-26 14:13:55   
ci-metacoq          14234  2021-06-05 20:15:10 2021-06-05 20:26:16   
                    13072  2021-06-09 15:18:55 2021-06-09 15:30:20   
                    14137  2021-06-14 09:42:50 2021-06-14 09:45:53   
ci-hott             13269  2021-06-21 01:38:40 2021-06-21 01:41:42   
ci-metacoq          14253  2021-06-21 19:07:57 2021-06-21 19:10:47   
ci-elpi             13107  2021-06-29 14:41:10 2021-06-29 14:44:05   
ci-metacoq          15128  2021-11-06 00:58:32 2021-11-06 01:02:56   
                    15390  2021-12-21 10:31:15 2021-12-21 10:35:27   
                    15518  2022-01-20 23:37:58 2022-01-20 23:41:23   
ci-category_theory  15518  2022-01-20 23:37:58 2022-01-20 23:41:38   
ci-sf               15518  2022-01-20 23:37:58 2022-01-20 23:41:13   
ci-itauto           15518  2022-01-20 23:37:58 2022-01-20 23:41:41   
ci-relation_algebra 15518  2022-01-20 23:37:58 2022-01-20 23:41:33   
ci-metacoq          15770  2022-03-05 08:12:33 2022-03-05 08:16:59   
ci-quickchick       15856  2022-03-25 20:42:21 2022-03-25 20:51:08   
ci-fiat_crypto      15949  2022-04-27 12:35:41 2022-04-27 12:45:34   
ci-category_theory  16322  2022-07-17 08:04:42 2022-07-17 08:09:19   

                                                                         body  \
target              number                                                      
ci-iris             13969   Error: Could not minimize file  (from ci-iris)...   
ci-metacoq          14234   Error: Could not minimize file  (from ci-metac...   
                    13072   Error: Could not minimize file  (from ci-metac...   
                    14137   Error: Could not minimize file  (from ci-metac...   
ci-hott             13269   Error: Could not minimize file  (from ci-hott)...   
ci-metacoq          14253   Error: Could not minimize file  (from ci-metac...   
ci-elpi             13107   Error: Could not minimize file  (from ci-elpi)...   
ci-metacoq          15128   Error: Could not minimize file  (from ci-metac...   
                    15390   Error: Could not minimize file  (from ci-metac...   
                    15518   Error: Could not minimize file  (from ci-metac...   
ci-category_theory  15518   Error: Could not minimize file  (from ci-categ...   
ci-sf               15518   Error: Could not minimize file  (from ci-sf) (...   
ci-itauto           15518   Error: Could not minimize file  (from ci-itaut...   
ci-relation_algebra 15518   Error: Could not minimize file  (from ci-relat...   
ci-metacoq          15770   Error: Could not minimize file  (from ci-metac...   
ci-quickchick       15856   Error: Could not minimize file /github/workspa...   
ci-fiat_crypto      15949   Error: Could not minimize file /github/workspa...   
ci-category_theory  16322   Error: Could not minimize file /github/workspa...   

                           targets  runtime  initial_size  final_size  \
target              number                                              
ci-iris             13969      NaN      NaN           NaN         NaN   
ci-metacoq          14234      NaN      NaN           NaN         NaN   
                    13072      NaN      NaN           NaN         NaN   
                    14137      NaN      NaN           NaN         NaN   
ci-hott             13269      NaN      NaN           NaN         NaN   
ci-metacoq          14253      NaN      NaN           NaN         NaN   
ci-elpi             13107      NaN      NaN           NaN         NaN   
ci-metacoq          15128      NaN      NaN           NaN         NaN   
                    15390      NaN      NaN           NaN         NaN   
                    15518      NaN      NaN           NaN         NaN   
ci-category_theory  15518      NaN      NaN           NaN         NaN   
ci-sf               15518      NaN      NaN           NaN        

Some comments were recorded on these errors in the tracking issue https://github.com/coq-community/run-coq-bug-minimizer/issues/8 or directly in the PR discussion.

|target | number | comment |
|-------|--------|---------|
| ci-mathcomp | 13969 | download issue? |
| ci-iris | 13969 | coqchk anomaly |
| ci-perennial | 14392 | base check not finished |
| ci-coq_performance_tests | 12743 | nondeterministic bug in Coq's native compilation handling, now a workaround is implemented |
| ci-metacoq | 14234 | minimization of metacoq did not work yet because of https://github.com/coq/coq/issues/14453 |
| ci-metacoq | 13072 | build system idempotency issue: ci-local target is not idempotent https://github.com/MetaCoq/metacoq/issues/563 |
| ci-verdi_raft | 13072 | issue with ambiguous libname prefixes https://github.com/JasonGross/coq-tools/issues/66 |
| ci-metacoq | 14137 | last case where the metacoq minimization did not work |
| ci-metacoq | 14253 | CI artifacts had expired |
| ci-elpi | 13107 | output test-suite fails with wrapper Coq binaries because file paths are absolutized https://github.com/math-comp/hierarchy-builder/issues/256 |
| ci-elpi | 14798 | same issue |
| ci-hott | 14929 | CI artifact download issue |
| ci-metacoq | 15128 | https://github.com/MetaCoq/metacoq/issues/605 |
| ci-metacoq | 15390 | same issue |
| ci-metacoq, ci-category_theory, ci-itauto, ci-sf, ci-relation_algebra | 15518 | the corresponding base jobs were skipped |

Finally, the runs which produced a minimized file but, from the header comments or their absence thereof, we can conclude that the minimization process failed to start (e.g., because it failed to reproduce the error message):

In [21]:
minimization_pairs[(minimization_pairs['success'] == False) & ~minimization_pairs['date_end'].isna() & minimization_pairs['body'].str.startswith('Minimized File')]

,,date_start,date_end,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success,duration
target,number,,,,,,,,,,,,
ci-equations,13969,2021-05-26 14:10:29,2021-05-26 14:14:19,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,230.0
ci-fourcolor,13969,2021-05-26 14:10:29,2021-05-26 14:15:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,298.0
ci-perennial,14392,2021-05-26 22:35:48,2021-05-26 23:04:20,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,1712.0
ci-hott,14492,2021-06-12 21:33:20,2021-06-12 21:38:57,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,337.0
ci-compcert,14598,2021-07-05 10:25:27,2021-07-05 10:41:26,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,959.0
ci-fiat_crypto_legacy,14671,2021-07-18 01:15:30,2021-07-18 01:22:03,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,393.0
ci-color,12493,2021-07-20 23:39:10,2021-07-20 23:44:43,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,333.0
ci-metacoq,11966,2021-07-25 16:58:47,2021-07-25 17:04:47,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,360.0
ci-vst,14783,2021-08-14 07:59:50,2021-08-14 08:04:51,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,301.0


Some comments were recorded on these errors in the tracking issue https://github.com/coq-community/run-coq-bug-minimizer/issues/8 or directly in the PR discussion.

|target | number | comment |
|-------|--------|---------|
| ci-equations | 13969 | mismatch between character locations and byte locations |
| ci-fourcolor | 13969 | `\r\n` line endings on Linux |
| ci-interval | 13895 | issue processing the error log (fixed) |
| ci-hott | 13269 | missing base artifacts |
| ci-fiat_parsers | 13072 | fixed in https://github.com/coq-community/run-coq-bug-minimizer/commit/38d5b1aedfc8944191276fc7be76b7c10a5e9fd9 |
| ci-hott | 14492 | no recognizable error? |
| ci-color | 12493 | failure because proxy constant names introduced by the PR change just from moving the Requires around |
| ci-equations | 13107 | error message parsing was too specific (fixed) |
| ci-cross_crypto | 13107 | namespace management issue (fixed) |
| ci-bedrock2 | 13107 | namespace management issue (fixed) |
| ci-coq_performance_tests | 13107 | issue using opam (fixed) |
| ci-tlc | 13107 | namespace management issue (fixed) |
| ci-perennial | 13107 | failure of run-minimizer to strip off -o arguments to coqc |
| ci-relation_algebra | 13107 | syntax error after stripping comments due a notation for ^* plus (x^*) showing up (fixed) |
| ci-fiat_crypto_legacy | 14671 | https://github.com/JasonGross/coq-tools/issues/107 |
| ci-metacoq | 11966 | https://github.com/JasonGross/coq-tools/issues/108 |
| ci-vst | 14783 | minimizer was misused by the user |
| ci-category_theory | 14818 | the error was independent from the PR |
| ci-category_theory | 12512 | Equations failed to install? |
| ci-cross_crypto | 14736 | probable failure of passing coqc due to eliminating Require after the buggy line |
| ci-fiat_crypto | 15128 | issue was that coqc ignores -top (fixed in the minimizer) |
| ci-bedrock2 | 15487 | Load results in miscomputed / misused glob file reported as https://github.com/coq/coq/issues/15497, fixed in the minimizer |
| ci-hott | 15518 | issue making universe names match in the error messages? |

#### RQ2: How often is this reduced test case fully standalone?

When are the reduced test cases fully standalone? We use the information from the header comments recording whether there was an inlining failure. However, this information was only added on 2021-10-21 (https://github.com/JasonGross/coq-tools/commit/d7217383349638adfba7c3a291a56237c37d84fc), so we only compute this for later runs.

In [22]:
runs_with_inlining_info = successful_minimization_pairs[successful_minimization_pairs['date_start'] >= pd.to_datetime('2021-10-21')]
no_inlining_failure = runs_with_inlining_info[runs_with_inlining_info['inline_failure'] == False]
len(runs_with_inlining_info)

106

In [23]:
len(no_inlining_failure) / len(runs_with_inlining_info)

0.8584905660377359

Let's take a look at the cases that were recorded with an inline failure:

In [24]:
runs_with_inlining_info[runs_with_inlining_info['inline_failure']==True]

date_start  duration  runtime  \
target                number                                          
ci-vst                15171  2021-11-12 11:13:07   19219.0   10.944   
                      11966  2021-12-03 17:25:58   22299.0    1.011   
ci-color              15400  2021-12-25 11:36:55    2370.0    1.018   
                      15501  2022-01-18 13:13:16   19204.0    1.399   
ci-gappa              15501  2022-01-18 13:13:16     902.0    0.484   
ci-geocoq             15518  2022-01-20 21:17:11   16415.0    1.084   
ci-iris               15657  2022-02-10 17:19:55   11955.0    0.910   
ci-bedrock2           15679  2022-02-14 18:18:39   23181.0    2.260   
ci-equations          15807  2022-03-15 15:47:12     360.0    0.125   
                      15822  2022-03-18 19:18:43     367.0    0.206   
ci-category_theory    16190  2022-06-27 22:49:05    1337.0    0.660   
ci-metacoq            16190  2022-06-27 22:49:05    8713.0   45.735   
ci-mtac2              16190  2022-06-27 22:49:05     627.0    0.477   
ci-jasmin             16311  2022-07-12 09:44:39   66952.0    3.122   
ci-fiat_crypto_legacy 16332  2022-07-20 13:24:05   11729.0    1.357   

                              initial_size  final_size inline_failure  \
target                number                                            
ci-vst                15171          330.0       613.0           True   
                      11966         2430.0      1320.0           True   
ci-color              15400          522.0        81.0           True   
                      15501          508.0       610.0           True   
ci-gappa              15501          727.0       105.0           True   
ci-geocoq             15518         2242.0       201.0           True   
ci-iris               15657           85.0       144.0           True   
ci-bedrock2           15679          116.0      1609.0           True   
ci-equations          15807          808.0        57.0           True   
                      15822          369.0        23.0           True   
ci-category_theory    16190          681.0        88.0           True   
ci-metacoq            16190          119.0        17.0           True   
ci-mtac2              16190           52.0       345.0           True   
ci-jasmin             16311         1590.0       876.0           True   
ci-fiat_crypto_legacy 16332          139.0       100.0           True   

                             truncated  total_line_reduction  
target                number                                  
ci-vst                15171      False                -234.0  
                      11966       True                1966.0  
ci-color              15400      False                 465.0  
                      15501      False                 181.0  
ci-gappa              15501      False                 664.0  
ci-geocoq             15518      False                2177.0  
ci-iris               15657      False                 104.0  
ci-bedrock2           15679      False               -1253.0  
ci-equations          15807      False                 757.0  
                      15822      False                 352.0  
ci-category_theory    16190      False                 671.0  
ci-metacoq            16190      False                 108.0  
ci-mtac2              16190      False                -253.0  
ci-jasmin             16311      False                1079.0  
ci-fiat_crypto_legacy 16332      False                 117.0

These failures to inline were diagnosed as such (and fixed):

|target | number | comment |
|-------|--------|---------|
| ci-vst | 15171 | the first inlining method failed and due to https://github.com/JasonGross/coq-tools/issues/110 (fixed), the other methods failed as well |
| ci-color | 15400 | robustness issue with error message https://github.com/JasonGross/coq-tools/issues/111 (fixed) |
| ci-color | 15501 | like ci-vst above, https://github.com/JasonGross/coq-tools/issues/110 |
| ci-gappa | 15501 | flocq not installing .glob files, fixed by https://github.com/coq/coq/pull/15509 |
| ci-geocoq | 15518 | like ci-vst above, https://github.com/JasonGross/coq-tools/issues/110 |

#### RQ3: How long does it take to produce such reduced test cases?

Duration for failed runs:

In [25]:
minimization_pairs[minimization_pairs['success'] == False]['duration'].describe(percentiles=[0.5,0.6,0.7,0.8,0.9,0.95])

count      33.000000
mean      387.515152
std       297.297633
min       170.000000
50%       301.000000
60%       324.200000
70%       373.200000
80%       493.400000
90%       651.400000
95%       794.600000
max      1712.000000
Name: duration, dtype: float64

Duration for successful runs:

In [26]:
successful_minimization_pairs['duration'].describe(percentiles=[0.5,0.6,0.7,0.8,0.9])

count      212.000000
mean      8480.528302
std      15318.031381
min        232.000000
50%       2057.500000
60%       3573.600000
70%       5376.100000
80%      13453.400000
90%      20217.300000
max      83501.000000
Name: duration, dtype: float64

#### RQ4: What is the size of the reduced cases?

In [27]:
no_inlining_failure['final_size'].describe()

count      91.000000
mean      505.846154
std      1065.592677
min         6.000000
25%        52.000000
50%       232.000000
75%       496.500000
max      9027.000000
Name: final_size, dtype: float64

In [28]:
successful_minimization_pairs['final_size'].describe()

count     212.000000
mean      413.443396
std       803.358933
min         3.000000
25%        44.750000
50%       151.500000
75%       478.250000
max      9027.000000
Name: final_size, dtype: float64

#### RQ5: How long do the reduced cases take to run?

For how many reduced cases do we have the expected runtime? What results do we get?

In [29]:
no_inlining_failure['runtime'].dropna().describe()

count    91.000000
mean      2.156165
std       7.066723
min       0.095000
25%       0.201000
50%       0.332000
75%       0.576500
max      48.066000
Name: runtime, dtype: float64

#### RQ6: What is the amount of code reduction?

How much smaller the final size is compared to the initial size?

In [30]:
np.mean(no_inlining_failure['final_size']/no_inlining_failure['initial_size'])

1.3031424339690183

In some cases, the final size can be larger than the initial size because of the dependency inlining process:

In [31]:
size_diffs = no_inlining_failure['initial_size']-no_inlining_failure['final_size']
len(size_diffs[size_diffs < 0])

17

In [32]:
len(size_diffs[size_diffs > 0])

74

Separate means for cases which were reduced and cases which were expanded:

In [33]:
np.mean(no_inlining_failure[size_diffs > 0]['final_size']/no_inlining_failure[size_diffs > 0]['initial_size'])

0.29224192670938454

In [34]:
np.mean(no_inlining_failure[size_diffs < 0]['final_size']/no_inlining_failure[size_diffs < 0]['initial_size'])

5.703532877334482

And when including all the dependencies that had to be inlined during the process?

In [35]:
np.mean(no_inlining_failure['final_size']/(no_inlining_failure['final_size'] + no_inlining_failure['total_line_reduction']))

0.6313530033489194

#### Project by project

This reveals significant differences but we have too little data to make any real conclusion:

In [36]:
successful_minimization_pairs.groupby(['target']).agg(['median','count'])

date_start       duration        \
                                          median count   median count   
target                                                                  
ci-aac_tactics           2021-07-07 22:08:10.000     1    232.0     1   
ci-analysis              2022-03-01 22:33:18.000     1  76278.0     1   
ci-argosy                2021-07-30 06:52:01.500     4   3163.5     4   
ci-bbv                   2021-07-07 22:08:10.000     3    619.0     3   
ci-bedrock2              2022-01-11 03:31:18.000    16   2204.5    16   
ci-bignums               2021-06-29 14:41:10.000     1   8529.0     1   
ci-category_theory       2021-12-17 14:27:39.000     5   1057.0     5   
ci-color                 2021-10-15 00:50:43.000     6  10486.0     6   
ci-compcert              2021-08-18 09:24:04.000     5   2156.0     5   
ci-coq_performance_tests 2021-07-03 02:20:06.000     3  19642.0     3   
ci-coqprime              2022-04-21 05:20:38.000     2   3819.0     2   
ci-coquelicot            2021-09-30 06:18:03.000     2    461.5     2   
ci-cross_crypto          2021-09-05 20:10:19.500     4   5018.0     4   
ci-elpi                  2021-12-27 00:34:48.500     2    419.0     2   
ci-equations             2021-09-06 17:53:40.000     5    338.0     5   
ci-fcsl_pcm              2022-01-01 01:10:13.000     2   2565.5     2   
ci-fiat_crypto           2021-11-12 11:13:07.000     7   1340.0     7   
ci-fiat_crypto_legacy    2021-08-24 00:44:25.000    12    791.5    12   
ci-fiat_parsers          2021-08-03 16:39:20.000    11    994.0    11   
ci-finmap                2022-03-05 08:12:33.000     1  26666.0     1   
ci-flocq                 2021-11-06 00:58:32.000     1    834.0     1   
ci-gappa                 2022-01-18 13:13:16.000     1    902.0     1   
ci-geocoq                2021-10-14 09:42:40.500     2  10291.5     2   
ci-hott                  2021-10-21 05:12:51.000    18   3218.0    18   
ci-interval              2021-05-27 22:11:13.000     1   2064.0     1   
ci-iris                  2022-01-12 22:08:29.000    12   4349.5    12   
ci-itauto                2022-01-18 13:13:16.000     1   2936.0     1   
ci-jasmin                2022-07-12 09:44:39.000     1  66952.0     1   
ci-math_classes          2021-08-03 12:29:14.000     4   1076.5     4   
ci-mathcomp              2021-09-22 06:58:56.000     6  27173.0     6   
ci-menhir                2022-03-01 09:52:57.000     4   1709.0     4   
ci-metacoq               2022-06-10 04:37:38.000     4  13945.5     4   
ci-mtac2                 2022-03-23 08:08:33.000     5    627.0     5   
ci-perennial             2021-12-21 10:31:15.000    13   8848.0    13   
ci-quickchick            2021-06-23 18:43:32.500     4    507.5     4   
ci-relation_algebra      2021-08-04 14:04:31.000     5   1176.0     5   
ci-rewriter              2021-08-04 03:21:55.500    10   1069.0    10   
ci-sf                    2022-01-12 17:46:07.500     2   2700.0     2   
ci-stdlib2               2021-08-13 12:21:08.000     1    373.0     1   
ci-tlc                   2021-07-07 22:08:10.000     3   4036.0     3   
ci-unimath               2021-08-06 14:10:50.000     5   2085.0     5   
ci-verdi_raft            2021-07-30 06:52:01.500     6    824.0     6   
ci-vst                   2021-11-23 02:19:32.500    10   9492.0    10   

                          runtime       initial_size       final_size        \
                           median count       median count     median count   
target                                                                        
ci-aac_tactics                NaN     0        279.0     1       27.0     1   
ci-analysis                4.2990     1       3941.0     1     9027.0     1   
ci-argosy                  0.7140     1        540.0     4      369.0     4   
ci-bbv                     0.0950     1       7591.0     3       33.0     3   
ci-bedrock2                0.3735    10        625.0    16      225.5    16   
ci-bignums                    NaN     0       1